# Problem Statement

- **Role:** You are the Learning & Development (L&D) Head.
- **Goal:** Create a system to generate a curated newsletter for employees.
- **Focus:** Latest advancements in AI Agents and Generative AI.
- **Resource:** Use Analytics Vidhya’s blogs for professional insights and updates.
- **System:** Develop an automated agentic system to:
    - Query Analytics Vidhya for recent articles and news.
    - Curate relevant content on AI Agents and Generative AI.
    - Automatically compile the curated content into a newsletter email.
- **Outcome:** Ensure employees stay informed and aligned with industry advancements, promoting continuous learning.

In [ ]:
#install tavily
# pip install tavily-python
#!pip install langchain
#!pip install langchain-community

In [1]:
import autogen
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
from autogen import register_function
from langchain_community.tools import TavilySearchResults
from autogen.interop import Interoperability
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

/opt/anaconda3/envs/ag2course/lib/python3.13/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
from dotenv import load_dotenv
load_dotenv('/Users/admin/Desktop/AutoGen/Module 1/.env')

True

# Sequential Chat with Tool Use

In [3]:
config_list = {
    "config_list": [{"model": "gpt-4o-mini", "temperature": 0.1}],
}

In [4]:
av_blog_agent = ConversableAgent(
    name="av_blog_agent",
    system_message="""You are a helpful AI assistant. You can use the given tool to get the relevant results.
                    Return 'TERMINATE' when the task is done.""",
    llm_config=config_list
)

In [5]:
user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="TERMINATE",
    code_execution_config=False
)

In [6]:
news_agent = ConversableAgent(
    name="news_agent",
    system_message="""You are a helpful AI assistant. You can use the given tool to get the relevant results.
                    Return 'TERMINATE' when the task is done.""",
    llm_config=config_list
)

In [7]:
tavily_tool = TavilySearchResults(max_results=5)

In [8]:
interop = Interoperability()
ag2_tool = interop.convert_tool(tool=tavily_tool, type="langchain")

In [9]:
ag2_tool.register_for_llm(news_agent)
ag2_tool.register_for_execution(user_proxy)

In [10]:
@av_blog_agent.register_for_llm(description="Searches Analytics Vidhya blog to get the results a for given query")
@user_proxy.register_for_execution()
def analytics_vidhya_search():
    """
    Extracts blog headings and descriptions from the given Analytics Vidhya blog archive URL.

    Returns:
        list: A list of dictionaries containing headings and descriptions.
    """

    response = requests.get("https://www.analyticsvidhya.com/blog-archive/")
    soup = BeautifulSoup(response.text, 'html.parser')

    list_items = soup.find_all('li', class_='row list-items')
    results = []

    for item in list_items:
        # Extract heading
        heading = item.find('h3', class_='fs-24 text-light-primary')
        heading_text = heading.text.strip() if heading else "No Heading"

        # Extract link
        url = heading.find_parent('a')['href'] if heading else "No Link"

        # Extract description
        description = item.find('p', class_='fs-16 text-dark-tertiary')
        description_text = description.text.strip() if description else "No Description"

        results.append({
            'heading': heading_text,
            'URL': url,
            'description': description_text,})

    return results

The return type of the function 'analytics_vidhya_search' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


**Step-by-Step Description:**

1. **Function Definition:**  
   `analytics_vidhya_search()` is defined to encapsulate the logic for retrieving and parsing blog data from the Analytics Vidhya blog archive.

2. **Decorators:**  
   - `@av_blog_agent.register_for_llm(description=...)`: Registers the function with an agent (`av_blog_agent`) that interacts with a language model. The description helps define the function's purpose.  
   - `@user_proxy.register_for_execution()`: Enables the function to be executed by the `user_proxy` agent, allowing it to be invoked as needed.

3. **HTTP Request:**  
   The function sends an HTTP `GET` request to the URL `"https://www.analyticsvidhya.com/blog-archive/"` using `requests.get()` to fetch the HTML content of the blog archive.

4. **Parsing HTML:**  
   The returned HTML is parsed using `BeautifulSoup`, creating a `soup` object that represents the DOM structure of the page.

5. **Locating Blog Entries:**  
   The function uses `soup.find_all('li', class_='row list-items')` to locate all `<li>` elements with the class `row list-items`. Each `<li>` corresponds to a blog entry containing a heading, URL, and description.

6. **Extracting Data:**  
   For each blog entry (`<li>` element):  
   - It retrieves the heading from an `<h3>` element with the class `fs-24 text-light-primary`. If no heading exists, it defaults to `"No Heading"`.  
   - It retrieves the URL from the parent `<a>` element of the heading. If no heading exists, it defaults to `"No Link"`.  
   - It retrieves the description from a `<p>` element with the class `fs-16 text-dark-tertiary`. If no description exists, it defaults to `"No Description"`.

7. **Building the Results:**  
   Each heading, URL, and description is stored in a dictionary and appended to the `results` list. This process creates a structured list of all the blog entries.

8. **Return Value:**  
   The function returns the `results` list, which contains dictionaries with `heading`, `URL`, and `description` for each blog entry. This structured output is ready for further processing or display.


In [11]:
email_agent = ConversableAgent(
    name="email_agent",
    system_message="""You can write effective news letter for the given content.
                    Return 'TERMINATE' when the task is done.""",
    llm_config=config_list
)

In [12]:
messages = ["Find the content on AI agents and include the URL in the content",
           """Find the latest news about the topics mentioned in the given content. Result should inlcude both initial content and news results""",
           "Write a news letter email for the given list of artilces and news."]

In [13]:
chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy,
            "recipient": av_blog_agent,
            "message": messages[0],
            "clear_history": True,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy,
            "recipient": news_agent,
            "message": messages[1],
            "summary_method": "last_msg",
        },
        {
            "sender": news_agent,
            "recipient": email_agent,
            "message": messages[2],
            "max_turns":1
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to av_blog_agent):

Find the content on AI agents and include the URL in the content

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
av_blog_agent (to User):

***** Suggested tool call (call_9M4T2KOMPZAfc5C0eJ7lSsya): analytics_vidhya_search *****
Arguments: 
{}
****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION analytics_vidhya_search...
User (to av_blog_agent):

***** Response from calling tool (call_9M4T2KOMPZAfc5C0eJ7lSsya) *****
[{"heading": "ChatGPT Just Got Smarter: Now You Can Schedule Tasks", "URL": "https://www.analyticsvidhya.com/blog/2025/01/sch

Please give feedback to av_blog_agent. Press enter or type 'exit' to stop the conversation:  exit



********************************************************************************
Starting a new chat....

********************************************************************************
User (to news_agent):

Find the latest news about the topics mentioned in the given content. Result should inlcude both initial content and news results
Context: 
Here are some articles related to AI agents:

1. **How to Build Agentic RAG with SmolAgents?**  
   Description: Build Agentic RAG with SmolAgents, creating intelligent AI systems capable of dynamic reasoning, retrieval, and multi-step task execution.  
   URL: [Read more](https://www.analyticsvidhya.com/blog/2025/01/agentic-rag-with-smolagents/)

2. **ChatGPT Just Got Smarter: Now You Can Schedule Tasks**  
   Description: Automate your daily workflow, set reminders, and stay organized with OpenAI's new feature - Scheduled Tasks in ChatGPT.  
   URL: [Read more](https://www.analyticsvidhya.com/blog/2025/01/schedule-tasks-with-chatgpt/)

3. 

Please give feedback to news_agent. Press enter or type 'exit' to stop the conversation:  exit



********************************************************************************
Starting a new chat....

********************************************************************************
news_agent (to email_agent):

Write a news letter email for the given list of artilces and news.
Context: 
Here are some articles related to AI agents:

1. **How to Build Agentic RAG with SmolAgents?**  
   Description: Build Agentic RAG with SmolAgents, creating intelligent AI systems capable of dynamic reasoning, retrieval, and multi-step task execution.  
   URL: [Read more](https://www.analyticsvidhya.com/blog/2025/01/agentic-rag-with-smolagents/)

2. **ChatGPT Just Got Smarter: Now You Can Schedule Tasks**  
   Description: Automate your daily workflow, set reminders, and stay organized with OpenAI's new feature - Scheduled Tasks in ChatGPT.  
   URL: [Read more](https://www.analyticsvidhya.com/blog/2025/01/schedule-tasks-with-chatgpt/)

3. **OpenAI’s New Function Calling Guide**  
   Description

In [14]:
display(Markdown(chat_results[2].summary))

Subject: Explore the Latest Innovations in AI Agents!

Dear [Recipient's Name],

We are excited to bring you the latest insights and developments in the world of AI agents! This newsletter features a selection of articles that delve into the functionalities and advancements of AI technologies, particularly focusing on SmolAgents, ChatGPT, and OpenAI's new features.

### Featured Articles:

1. **How to Build Agentic RAG with SmolAgents?**  
   Discover how to create intelligent AI systems capable of dynamic reasoning, retrieval, and multi-step task execution using SmolAgents.  
   [Read more](https://www.analyticsvidhya.com/blog/2025/01/agentic-rag-with-smolagents/)

2. **ChatGPT Just Got Smarter: Now You Can Schedule Tasks**  
   Learn about OpenAI's new feature that allows you to automate your daily workflow, set reminders, and stay organized with Scheduled Tasks in ChatGPT.  
   [Read more](https://www.analyticsvidhya.com/blog/2025/01/schedule-tasks-with-chatgpt/)

3. **OpenAI’s New Function Calling Guide**  
   This guide empowers developers to extend OpenAI models with best practices and examples for function calling.  
   [Read more](https://www.analyticsvidhya.com/blog/2025/01/openai-function-calling-guide/)

### Latest News Highlights:

#### 1. How to Build Agentic RAG with SmolAgents?
- **[SmolAgents Overview](https://www.analyticsvidhya.com/blog/2025/01/smolagents/)**: An introduction to the foundational building blocks for creating dynamic agentic systems.
- **[Building Agentic RAG](https://medium.com/the-ai-forum/build-an-agentic-rag-using-huggingface-transformer-agent-ec741f09ddcc)**: A detailed discussion on using a retriever tool for knowledge-based question answering.
- **[Hugging Face RAG Example](https://huggingface.co/docs/smolagents/examples/rag)**: Insights on using semantic search for retrieving relevant documentation.

#### 2. ChatGPT Just Got Smarter: Now You Can Schedule Tasks
- **[ChatGPT Scheduled Tasks Feature](https://www.cnet.com/tech/services-and-software/chatgpt-can-now-be-set-up-to-do-scheduled-tasks/)**: Explore the new feature that allows users to set up recurring actions and reminders.
- **[Managing Scheduled Tasks](https://help.openai.com/en/articles/10291617-scheduled-tasks-in-chatgpt)**: A guide on creating and managing scheduled tasks within ChatGPT.
- **[Beta Feature Announcement](https://techcrunch.com/2025/01/14/chatgpt-now-lets-you-schedule-reminders-and-recurring-tasks/)**: OpenAI's announcement regarding the beta feature for scheduling reminders.

#### 3. OpenAI’s New Function Calling Guide
- **[Function Calling Guide](https://www.analyticsvidhya.com/blog/2025/01/openai-function-calling-guide/)**: Learn how to extend model capabilities with custom tools and functions.
- **[Structured Outputs in Function Calling](https://openai.com/index/function-calling-and-other-api-updates/)**: Discover how developers can describe functions for structured JSON outputs.
- **[Using Function Calling](https://metaschool.so/articles/openai-function-calling)**: A practical explanation of utilizing OpenAI's function calling feature for structured outputs.

We hope you find these articles informative and inspiring as you explore the evolving landscape of AI agents. Stay tuned for more updates!

Best regards,  
[Your Name]  
[Your Position]  
[Your Company]  
[Your Contact Information]

